# Actor

In [ ]:
import pandas as pd
import json
import random

credits = pd.read_csv("credits.csv", low_memory=False)

# Función para expandir columnas JSON en múltiples filas
def expand_json_column(df, column_name, id_column):
    expanded_data = []
    for _, row in df.iterrows():
        try:
            json_list = json.loads(row[column_name].replace("'", "\"")) if pd.notna(row[column_name]) else []
            for item in json_list:
                item_data = item.copy()
                item_data[id_column] = row[id_column]
                expanded_data.append(item_data)
        except json.JSONDecodeError:
            continue

    return pd.DataFrame(expanded_data)

# Asegurar que 'cast' es string y manejar valores NaN
credits['cast'] = credits['cast'].astype(str).fillna('[]')

# Expandir la columna JSON 'cast' desde credits.csv
actors_expanded = expand_json_column(credits, 'cast', 'id')

# Seleccionar solo las columnas necesarias y eliminar duplicados
actors_cleaned = actors_expanded[['cast_id', 'name', 'gender', 'character', 'id']].drop_duplicates()
actors_cleaned.rename(columns={
    'cast_id': 'ActorID',
    'name': 'Name',
    'gender': 'Gender',
    'character': 'Role',
    'id': 'MovieID'  # Relación con la película
}, inplace=True)

# Generar fechas de nacimiento ficticias para los actores (entre 1950 y 2005)
random.seed(42)
actors_cleaned['Born_Date'] = [random.randint(1950, 2005) for _ in range(len(actors_cleaned))]

# Guardar el CSV de actores
actor_csv_path = "Actor_LIMPIO.csv"
actors_cleaned.to_csv(actor_csv_path, index=False)



In [ ]:
# Crear un nuevo id para cada actor basado en su nombre
actors_cleaned['ActorID'] = actors_cleaned['Name'].astype('category').cat.codes

# Volver a guardar el CSV con el nuevo id
actor_csv_path = "Actor_LIMPIO2.csv"
actors_cleaned.to_csv(actor_csv_path, index=False)



# Movie

In [ ]:
movies = pd.read_csv("movies_metadata.csv", low_memory=False)
keywords = pd.read_csv("keywords.csv", low_memory=False)

# Seleccionar solo las columnas necesarias de movies_metadata
movies_selected = movies[['id', 'title', 'release_date', 'runtime', 'popularity']].copy()

# Renombrar columnas según nuestro esquema
movies_selected.rename(columns={
    'id': 'MovieID',
    'title': 'Title',
    'release_date': 'Release_Date',
    'runtime': 'Runtime',
    'popularity': 'Popularity'
}, inplace=True)

# Limpiar valores nulos o incorrectos
movies_selected['Release_Date'] = pd.to_datetime(movies_selected['Release_Date'], errors='coerce')  # Convertir a fecha
movies_selected['Runtime'] = pd.to_numeric(movies_selected['Runtime'], errors='coerce').fillna(0)  # Convertir a numérico
movies_selected['Popularity'] = pd.to_numeric(movies_selected['Popularity'], errors='coerce').fillna(0)  # Convertir a numérico

# Asegurar que 'id' en keywords es string para evitar errores al unir
keywords['id'] = keywords['id'].astype(str)
movies_selected['MovieID'] = movies_selected['MovieID'].astype(str)

# Unir keywords con movies_metadata usando MovieID
movies_final = movies_selected.merge(keywords[['id', 'keywords']], left_on='MovieID', right_on='id', how='left')
movies_final.drop(columns=['id'], inplace=True)  # Eliminar columna redundante

# Renombrar la columna de keywords
movies_final.rename(columns={'keywords': 'Keywords'}, inplace=True)

# Guardar el CSV de películas actualizado
movie_csv_path = "Movie_LIMPIO2.csv"
movies_final.to_csv(movie_csv_path, index=False)




In [ ]:
# Función para extraer solo los nombres de keywords desde JSON
def extract_keyword_names(json_str):
    try:
        keywords_list = json.loads(json_str.replace("'", "\"")) if pd.notna(json_str) else []
        return ", ".join([kw["name"] for kw in keywords_list])  # Convertir a string separado por comas
    except json.JSONDecodeError:
        return ""

# Aplicar la función a la columna de keywords
movies_final['Keywords'] = movies_final['Keywords'].astype(str).apply(extract_keyword_names)

# Guardar el CSV corregido
movie_csv_path = "Movie_LIMPIO3.csv"
movies_final.to_csv(movie_csv_path, index=False)




In [ ]:
import pandas as pd
import json

# Cargar archivos
movies = pd.read_csv("movies_metadata.csv", low_memory=False)
keywords = pd.read_csv("keywords.csv", low_memory=False)

# Seleccionar solo las columnas necesarias de movies_metadata
movies_selected = movies[['id', 'title', 'release_date', 'runtime', 'popularity']].copy()

# Renombrar columnas según nuestro esquema
movies_selected.rename(columns={
    'id': 'MovieID',
    'title': 'Title',
    'release_date': 'Release_Date',
    'runtime': 'Runtime',
    'popularity': 'Popularity'
}, inplace=True)

# Limpiar valores nulos o incorrectos
movies_selected['Release_Date'] = pd.to_datetime(movies_selected['Release_Date'], errors='coerce')  # Convertir a fecha
movies_selected['Runtime'] = pd.to_numeric(movies_selected['Runtime'], errors='coerce').fillna(0)  # Convertir a numérico
movies_selected['Popularity'] = pd.to_numeric(movies_selected['Popularity'], errors='coerce').fillna(0)  # Convertir a numérico

# Verificar el formato de los IDs en ambos archivos
keywords['id'] = keywords['id'].astype(str)
movies_selected['MovieID'] = movies_selected['MovieID'].astype(str)

# Unir keywords con movies_metadata usando MovieID
movies_final = movies_selected.merge(keywords, left_on='MovieID', right_on='id', how='left')

# Función para extraer solo los nombres de keywords desde JSON
def extract_keyword_names(json_str):
    """Extrae solo los nombres de las keywords desde un string JSON."""
    try:
        keywords_list = json.loads(json_str.replace("'", "\"")) if pd.notna(json_str) else []
        return ", ".join([kw["name"] for kw in keywords_list])  # Convertir a string separado por comas
    except json.JSONDecodeError:
        return ""

# Aplicar la función a la columna de keywords
movies_final['keywords'] = movies_final['keywords'].fillna('[]').astype(str)
movies_final['keywords'] = movies_final['keywords'].apply(extract_keyword_names)

# Renombrar la columna de keywords
movies_final.rename(columns={'keywords': 'Keywords'}, inplace=True)

# Eliminar la columna 'id' redundante después del merge
movies_final.drop(columns=['id'], inplace=True)

# Guardar el CSV corregido
movie_csv_path = "Movie_LIMPIO4.csv"
movies_final.to_csv(movie_csv_path, index=False)




# Director

In [ ]:
credits = pd.read_csv("credits.csv")

# Extraer solo las filas donde la persona tiene el rol de "Director" en la columna crew
credits['crew'] = credits['crew'].fillna('[]')

# Función para extraer directores desde JSON
def extract_directors(json_str, movie_id):
    """Extrae directores desde la columna crew en formato JSON."""
    try:
        crew_list = json.loads(json_str.replace("'", "\"")) if pd.notna(json_str) else []
        return [
            {"DirectorID": crew["id"], "Name": crew["name"], "Gender": crew["gender"], "MovieID": movie_id}
            for crew in crew_list if crew.get("job") == "Director"
        ]
    except json.JSONDecodeError:
        return []

# Aplicar la función a todas las filas del DataFrame
directors_data = []
for _, row in credits.iterrows():
    directors_data.extend(extract_directors(row['crew'], row['id']))

# Convertir la lista de directores en un DataFrame
directors_cleaned = pd.DataFrame(directors_data)

# Agregar columna de edad inventada (entre 30 y 75 años)
import numpy as np
np.random.seed(42)  # Para reproducibilidad
directors_cleaned['Age'] = np.random.randint(30, 75, size=len(directors_cleaned))

# Agregar columna de premios aleatorios (entre 0 y 10, con tendencia a números bajos)
directors_cleaned['Awards'] = np.random.choice([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10], size=len(directors_cleaned), p=[0.3, 0.2, 0.15, 0.1, 0.1, 0.05, 0.04, 0.03, 0.02, 0.01, 0.00])

# Guardar el CSV de directores
director_csv_path = "Director_FINAL.csv"
directors_cleaned.to_csv(director_csv_path, index=False)




# Country

In [ ]:
import pandas as pd
import json
import numpy as np


movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# Asegurar que 'production_countries' sea string y reemplazar valores NaN correctamente
movies['production_countries'] = movies['production_countries'].apply(lambda x: "[]" if pd.isna(x) else str(x))

# Función para extraer países con su formato
def extract_countries_safe(json_str, movie_id):
    try:
        country_list = json.loads(json_str.replace("'", "\"")) if json_str.strip() not in ["", "[]", "nan"] else []
        return [{"CountryID": country.get("iso_3166_1", "Unknown"), "Name": country.get("name", "Unknown"), "MovieID": movie_id} for country in country_list]
    except (json.JSONDecodeError, TypeError):
        return []

# Extraer datos de países
countries_data_safe = []
for _, row in movies.iterrows():
    countries_data_safe.extend(extract_countries_safe(row['production_countries'], row['id']))

# Convertir a df y eliminar duplicados
countries_cleaned_safe = pd.DataFrame(countries_data_safe).drop_duplicates()

# Datos ficticios para población, idioma y ciudades
population_data = {
    "United States": 331000000, "United Kingdom": 67000000, "France": 65000000, "Germany": 83000000, "Canada": 38000000,
    "Australia": 25000000, "India": 1390000000, "Japan": 126000000, "Mexico": 126000000, "Brazil": 213000000
}

language_data = {
    "United States": "English", "United Kingdom": "English", "France": "French", "Germany": "German", "Canada": "English, French",
    "Australia": "English", "India": "Hindi, English", "Japan": "Japanese", "Mexico": "Spanish", "Brazil": "Portuguese"
}

cities_data = {
    "United States": "New York, Los Angeles, Chicago", "United Kingdom": "London, Manchester, Birmingham",
    "France": "Paris, Marseille, Lyon", "Germany": "Berlin, Munich, Hamburg", "Canada": "Toronto, Vancouver, Montreal",
    "Australia": "Sydney, Melbourne, Brisbane", "India": "Mumbai, Delhi, Bangalore", "Japan": "Tokyo, Osaka, Kyoto",
    "Mexico": "Mexico City, Guadalajara, Monterrey", "Brazil": "São Paulo, Rio de Janeiro, Brasília"
}

# Asignar datos adicionales basados en el nombre del país
countries_cleaned_safe['Population'] = countries_cleaned_safe['Name'].map(population_data).fillna(5000000)
countries_cleaned_safe['Language'] = countries_cleaned_safe['Name'].map(language_data).fillna("Unknown")
countries_cleaned_safe['Cities'] = countries_cleaned_safe['Name'].map(cities_data).fillna("Unknown")

# Guardar el archivo corregido
country_csv_path_safe = "Country_FINAL3.csv"
countries_cleaned_safe.to_csv(country_csv_path_safe, index=False)


    


# USER

In [30]:
import pandas as pd
import numpy as np
import random

# Generar 50 nombres masculinos y 50 femeninos
male_names = ["John", "Michael", "David", "James", "Robert", "William", "Joseph", "Charles", "Thomas", "Daniel",
              "Matthew", "Anthony", "Mark", "Donald", "Steven", "Paul", "Andrew", "Joshua", "Kenneth", "Kevin",
              "Brian", "George", "Edward", "Ronald", "Timothy", "Jason", "Jeffrey", "Ryan", "Jacob", "Gary",
              "Nicholas", "Eric", "Jonathan", "Stephen", "Larry", "Justin", "Scott", "Brandon", "Benjamin", "Samuel",
              "Gregory", "Frank", "Alexander", "Raymond", "Patrick", "Jack", "Dennis", "Jerry", "Tyler", "Aaron"]

female_names = ["Mary", "Patricia", "Jennifer", "Linda", "Elizabeth", "Barbara", "Susan", "Jessica", "Sarah", "Karen",
                "Nancy", "Margaret", "Lisa", "Betty", "Dorothy", "Sandra", "Ashley", "Kimberly", "Emily", "Donna",
                "Michelle", "Carol", "Amanda", "Melissa", "Deborah", "Stephanie", "Rebecca", "Sharon", "Laura", "Cynthia",
                "Kathleen", "Amy", "Shirley", "Angela", "Helen", "Anna", "Brenda", "Pamela", "Nicole", "Samantha",
                "Katherine", "Emma", "Ruth", "Christine", "Catherine", "Debra", "Rachel", "Carolyn", "Janet", "Maria"]

# Crear lista de usuarios
user_data = []
user_id = 1

def generate_ratings(user_id, name, gender, age, movie_ids):
    num_ratings = np.random.randint(2, 51)  # Entre 2 y 50 ratings por usuario
    rated_movies = np.random.choice(movie_ids, size=num_ratings, replace=False)
    ratings = np.round(np.random.uniform(1, 5, size=num_ratings), 2)  # Ratings entre 1 y 5
    return [(user_id, name, gender, age, movie_id, rating) for movie_id, rating in zip(rated_movies, ratings)]

# Obtener MovieIDs únicos
movies = pd.read_csv("movies_metadata.csv", low_memory=False)
movies = movies.dropna(subset=["id"])
movies["id"] = movies["id"].astype(str).str.extract(r'(\d+)').astype(float).dropna().astype(int)
movie_ids = movies["id"].unique()

# Asignar usuarios con ratings
for i, name in enumerate(male_names + female_names):
    gender = "Male" if i < 50 else "Female"
    age = np.random.randint(18, 65)  # Generar edades entre 18 y 65 años
    ratings = generate_ratings(user_id, name, gender, age, movie_ids)
    user_data.extend(ratings)
    user_id += 1

# Convertir a DataFrame y guardar
users_df = pd.DataFrame(user_data, columns=["UserID", "Name", "Gender", "Age", "MovieID", "Rating"])
users_df.to_csv("User_FINAL4.csv", index=False)




# Genre

In [ ]:
import pandas as pd
import json

movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# Asegurar que la columna 'genres' no tenga valores nulos
movies['genres'] = movies['genres'].fillna('[]')

# Extraer los géneros únicos
genres_data = []
for _, row in movies.iterrows():
    try:
        genre_list = json.loads(row['genres'].replace("'", "\"")) if pd.notna(row['genres']) else []
        for genre in genre_list:
            genres_data.append({"GenreID": genre["id"], "Name": genre["name"]})
    except json.JSONDecodeError:
        continue

# Convertir a DataFrame y eliminar duplicados
genres_df = pd.DataFrame(genres_data).drop_duplicates().reset_index(drop=True)

# Guardar como CSV
genre_csv_path = "Genre_FINAL.csv"
genres_df.to_csv(genre_csv_path, index=False)


# Company

In [ ]:
import pandas as pd
import numpy as np

movies = pd.read_csv("movies_metadata.csv", low_memory=False)

# Expandir la columna JSON 'production_companies'
import json

def extract_companies(json_str, movie_id):
    """Extrae la lista de compañías de producción desde JSON."""
    try:
        company_list = json.loads(json_str.replace("'", "\"")) if pd.notna(json_str) else []
        return [{"CompanyID": company["id"], "Name": company["name"], "MovieID": movie_id} for company in company_list]
    except json.JSONDecodeError:
        return []

companies_data = []
for _, row in movies.iterrows():
    companies_data.extend(extract_companies(row['production_companies'], row['id']))

# Convertir a df
companies_df = pd.DataFrame(companies_data)

# Asegurar que no haya IDs duplicados asignando CompanyID únicos
companies_df.drop_duplicates(subset=['CompanyID'], inplace=True)

# Generar datos ficticios para estudio, número de trabajadores y año de fundación
np.random.seed(42)
companies_df['Studio'] = np.random.choice(["Warner Bros", "Universal Pictures", "Paramount", "Sony Pictures", "20th Century Fox"], size=len(companies_df))
companies_df['Num_Workers'] = np.random.randint(50, 10000, size=len(companies_df))
companies_df['CreatedIn'] = np.random.randint(1920, 2022, size=len(companies_df))

# Guardar en CSV
company_csv_path = "Company_FINAL.csv"
companies_df.to_csv(company_csv_path, index=False)




# Modificar Movie CSV

In [ ]:
import pandas as pd
import numpy as np
import json

# Cargar datos base
movies = pd.read_csv("movies_metadata.csv", low_memory=False)
credits = pd.read_csv("credits.csv", low_memory=False)
keywords = pd.read_csv("keywords.csv", low_memory=False)

# Convertir MovieID a tipo entero donde sea posible
movies["id"] = pd.to_numeric(movies["id"], errors="coerce").astype("Int64")
credits["id"] = pd.to_numeric(credits["id"], errors="coerce").astype("Int64")
keywords["id"] = pd.to_numeric(keywords["id"], errors="coerce").astype("Int64")

# Función para extraer JSON y normalizarlo
def extract_json_data(df, column_name, id_column):
    extracted_data = []
    for _, row in df.iterrows():
        try:
            if pd.notna(row[column_name]) and isinstance(row[column_name], str):
                json_obj = json.loads(row[column_name].replace("'", "\""))
                for obj in json_obj:
                    obj[id_column] = row[id_column]
                    extracted_data.append(obj)
        except (json.JSONDecodeError, TypeError):
            continue
    return pd.DataFrame(extracted_data)

# Obtener DirectorID desde el dataset credits.csv
credits["crew"] = credits["crew"].fillna("[]")
crew_data = extract_json_data(credits, "crew", "id")
directors = crew_data[crew_data["job"] == "Director"][["id", "name"]].rename(columns={"id": "MovieID", "name": "DirectorID"})

# Obtener CountryID desde el dataset movies_metadata.csv
movies["production_countries"] = movies["production_countries"].fillna("[]")
countries_data = extract_json_data(movies, "production_countries", "id")
countries_data = countries_data.rename(columns={"iso_3166_1": "CountryID", "id": "MovieID"})

# Obtener GenreID desde el dataset movies_metadata.csv
movies["genres"] = movies["genres"].fillna("[]")
genres_data = extract_json_data(movies, "genres", "id")
genres_data = genres_data.rename(columns={"id": "MovieID", "name": "GenreID"})

# Obtener CompanyID desde el dataset movies_metadata.csv
movies["production_companies"] = movies["production_companies"].fillna("[]")
companies_data = extract_json_data(movies, "production_companies", "id")
companies_data = companies_data.rename(columns={"id": "MovieID", "name": "CompanyID"})

# Obtener Keywords desde el dataset keywords.csv
keywords["keywords"] = keywords["keywords"].fillna("[]")
keywords_data = extract_json_data(keywords, "keywords", "id")
keywords_data = keywords_data.rename(columns={"id": "MovieID", "name": "Keyword"})

# Seleccionar columnas necesarias para Movie.csv
movies_selected = movies[["id", "title", "release_date", "runtime", "popularity"]].copy()
movies_selected.rename(columns={
    "id": "MovieID",
    "title": "Title",
    "release_date": "Released",
    "runtime": "Runtime",
    "popularity": "Popularity"
}, inplace=True)

# Agregar las relaciones a Movie.csv
movies_final = movies_selected.merge(directors, on="MovieID", how="left")
movies_final = movies_final.merge(countries_data[["MovieID", "CountryID"]], on="MovieID", how="left")
movies_final = movies_final.merge(genres_data[["MovieID", "GenreID"]], on="MovieID", how="left")
movies_final = movies_final.merge(companies_data[["MovieID", "CompanyID"]], on="MovieID", how="left")

# Agregar Keywords (se agrupan todas las palabras clave de cada película en una lista separada por comas)
keywords_aggregated = keywords_data.groupby("MovieID")["Keyword"].apply(lambda x: ", ".join(x)).reset_index()
movies_final = movies_final.merge(keywords_aggregated, on="MovieID", how="left")

# Guardar Movie_Final.csv
movies_final.to_csv("Movie_Final_7.csv", index=False)


## Modificar el ID de género en la tabla de movies

In [1]:
import pandas as pd

genres_df = pd.read_csv('Genre_FINAL.csv')
movies_df = pd.read_csv('Movie_Final_7.csv')

genre_mapping = dict(zip(genres_df['Name'], genres_df['GenreID']))

# Reemplazamos en movies_df la columna de género por el id correspondiente
movies_df['GenreID'] = movies_df['GenreID'].map(genre_mapping)

# Hacer nuevo archivo
movies_df.to_csv('Movie_FINAL_8.csv', index=False)

print("El archivo movies.csv ha sido actualizado con los IDs de género.")


/var/folders/tr/4sj_ng597kdbn8vp_yc6jltm0000gn/T/ipykernel_91458/1669657304.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('Movie_Final_7.csv')


El archivo movies.csv ha sido actualizado con los IDs de género.


## Agregar columnas

In [2]:
import pandas as pd

genres_df = pd.read_csv('Genre_FINAL.csv')
movies_df = pd.read_csv('Movie_FINAL_8.csv')

# Conta el numero de veces que aparece el id en movies
num_movies = movies_df['GenreID'].value_counts().to_dict()

# Se crea la nueva columna
genres_df['numMovies'] = genres_df['GenreID'].map(num_movies).fillna(0).astype(int)

# Calcular el valor máximo para escalar la popularidad
max_count = genres_df['numMovies'].max()

# Calcular la columna 'popularity' de forma que el género con mayor cantidad tenga 5 y el resto se escale proporcionalmente.
# Se usa una función lambda que asigna al menos 1
if max_count == 0:
    genres_df['popularity'] = 1
else:
    genres_df['popularity'] = genres_df['numMovies'].apply(lambda x: max(1, round((x / max_count) * 5)))

# Agregar la columna 'classification' vacía
genres_df['classification'] = ""

# Sobrescribir el archivo genre.csv con el DataFrame actualizado
genres_df.to_csv('Genre_FINAL_2.csv', index=False)

print("El archivo genre.csv ha sido actualizado con las columnas 'popularity', 'numMovies' y 'classification'.")


/var/folders/tr/4sj_ng597kdbn8vp_yc6jltm0000gn/T/ipykernel_91458/3135294082.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('Movie_FINAL_8.csv')


El archivo genre.csv ha sido actualizado con las columnas 'popularity', 'numMovies' y 'classification'.


## Agregar el CountryID a company

In [4]:
import pandas as pd

company_df = pd.read_csv('Company_FINAL.csv')
movies_df = pd.read_csv('Movie_FINAL_8.csv')

# Realizar el merge para obtener la columna 'CountryID' del archivo de películas
# Se asume que en movies.csv las columnas relevantes son 'movieID' y 'CountryID'
company_df = company_df.merge(movies_df[['MovieID', 'CountryID']], on='MovieID', how='left')

# Guardar los cambios sobrescribiendo el archivo company.csv
company_df.to_csv('Company_FINAL_2.csv', index=False)

print("El archivo company.csv ha sido actualizado con la columna 'CountryID'.")


/var/folders/tr/4sj_ng597kdbn8vp_yc6jltm0000gn/T/ipykernel_91458/3312646231.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('Movie_FINAL_8.csv')


El archivo company.csv ha sido actualizado con la columna 'CountryID'.


## Agregar amigo

In [5]:
import pandas as pd
import numpy as np

user_df = pd.read_csv('User_FINAL4.csv')

# Crear una lista con todos los UserID
user_ids = user_df['UserID'].tolist()

# Función para elegir un amigo aleatorio, excluyendo el usuario actual
def choose_friend(current_user, user_ids):
    # Lista de posibles amigos
    available_ids = [uid for uid in user_ids if uid != current_user]
    return np.random.choice(available_ids) if available_ids else None

#crear la nueva columna
user_df['friendsUser'] = user_df['UserID'].apply(lambda x: choose_friend(x, user_ids))

user_df.to_csv('User_FINAL5.csv', index=False)

print("El archivo user.csv ha sido actualizado con la columna 'friendsUser'.")


El archivo user.csv ha sido actualizado con la columna 'friendsUser'.


In [6]:
import pandas as pd
import numpy as np

# Cargar el archivo user.csv
user_df = pd.read_csv('User_FINAL4.csv')

# Extraer la lista de todos los UserID
user_ids = user_df['UserID'].tolist()

# Función para elegir un amigo aleatorio, excluyendo el usuario actual
def choose_friend(current_user, user_ids):
    # Crear la lista de posibles amigos sin incluir el usuario actual
    available_ids = [uid for uid in user_ids if uid != current_user]
    return np.random.choice(available_ids) if available_ids else None

# Crear la nueva columna 'friendsUser' aplicando la función a cada UserID
user_df['friendsUser'] = user_df['UserID'].apply(lambda x: choose_friend(x, user_ids))

# Crear un nuevo DataFrame solo con las columnas 'UserID' y 'friendsUser'
result_df = user_df[['UserID', 'friendsUser']]

# Guardar el nuevo DataFrame en un archivo CSV
result_df.to_csv('user_friends.csv', index=False)

print("Se ha creado el archivo 'user_friends.csv' con las columnas 'UserID' y 'friendsUser'.")


Se ha creado el archivo 'user_friends.csv' con las columnas 'UserID' y 'friendsUser'.
